# Pollution in Indian cities 2024.
USE JANUARY FROM 2024
Dowload from each city individually. Merge together
Tripura state not there as data goes to 2023 only.

In [1]:
import os
import pandas as pd
import numpy as np



In [2]:
os.getcwd()

'/Users/elenalarrauri/LSE/COURSES/AT/Managing_data/2024-project-pollution-team'

# Test for a single excel file.
There are NaN values and NA values, we will have to switch NA to NaN

In [3]:
# To avoid a warning.
pd.set_option("future.no_silent_downcasting", True)


In [4]:
pollution_city = pd.read_excel("pollution_cities/andhra pradesh/AQI_daily_city_level_amaravati_2024_amaravati_2024.xlsx",nrows=31)

In [5]:
value_vars = pollution_city.columns.to_list()
value_vars.remove("Date")
value_vars

['January',
 'February',
 'March',
 'April',
 'May',
 'June',
 'July',
 'August',
 'September',
 'October',
 'November',
 'December']

In [6]:
VALUE_VARS = tuple(value_vars)

In [7]:
#"pollution_cities/andhra pradesh/AQI_daily_city_level_amaravati_2024_amaravati_2024.xlsx"
def read_excel(path,city,state,value_vars=VALUE_VARS):
    """
    Takes excel for different cities and cleans and converts to long format
    """
    
    pollution_city = pd.read_excel(path,nrows=31)
    
    pollution_city.replace('NA ', np.nan, inplace=True)
    # Just in case
    pollution_city.replace('NA', np.nan, inplace=True)
    
    pollution_city_long = pollution_city.melt(id_vars="Date",value_vars=value_vars)
    
    pollution_city_long["value"] = pd.to_numeric(pollution_city_long["value"])

    # Add state and city identifier.
    pollution_city_long["city"] = city
    pollution_city_long["state"] = state

    return pollution_city_long
    
    
    

In [8]:
read_excel("pollution_cities/andhra pradesh/AQI_daily_city_level_amaravati_2024_amaravati_2024.xlsx","amaravati","andhra pradesh")

,Date,variable,value,city,state
0,1,January,241.0,amaravati,andhra pradesh
1,2,January,224.0,amaravati,andhra pradesh
2,3,January,261.0,amaravati,andhra pradesh
3,4,January,241.0,amaravati,andhra pradesh
4,5,January,151.0,amaravati,andhra pradesh
...,...,...,...,...,...
367,27,December,NaN,amaravati,andhra pradesh
368,28,December,NaN,amaravati,andhra pradesh
369,29,December,NaN,amaravati,andhra pradesh
370,30,December,NaN,amaravati,andhra pradesh


# Loop over multiple cities.

In [9]:
test_str = 'AQI_daily_city_level_chittoor_2024_chittoor_2024.xlsx'

In [10]:
# https://www.geeksforgeeks.org/python-extract-string-between-two-substrings/

def city_name(test_str):
    """
    
    Get the city name from the name of the excel file
    
    """
    idx1 = test_str.index("AQI_daily_city_level_")
    idx2 = test_str.index("_2024_")
     
    # length of substring 1 is added to
    # get string from next character
    res = test_str[idx1 + len("AQI_daily_city_level_"): idx2]
    
    return res

In [11]:
city_name(test_str)

'chittoor'

In [12]:
list_df = []

In [13]:
# Use os to loop over every file in the directory.
counter = 0
for state in os.listdir("pollution_cities"):
    if state != ".DS_Store":
        
        # Go through every city in each state.
        
        for city in os.listdir(f"pollution_cities/{state}"):
            
            if city != ".DS_Store" and city != ".ipynb_checkpoints":
                # Count the number of cities.
                counter +=1
                
                # For each city extract the name, using the function defined above.
                #print(city,state)
                name_city = city_name(city)

                # Read the excel file.
                full_path = "pollution_cities/"+state+"/"+city

                # append the cleaned city dataframe.
                list_df.append(read_excel(path=full_path,city=name_city,state=state))
                
                
        
        
        
                

In [14]:
# Remarkably well formatted across cities, so nice :)
df = pd.concat(list_df
         )



In [15]:
df

,Date,variable,value,city,state
0,1,January,NaN,naharlagun,arunachal pradesh
1,2,January,NaN,naharlagun,arunachal pradesh
2,3,January,NaN,naharlagun,arunachal pradesh
3,4,January,NaN,naharlagun,arunachal pradesh
4,5,January,NaN,naharlagun,arunachal pradesh
...,...,...,...,...,...
367,27,December,NaN,kohima,nagaland
368,28,December,NaN,kohima,nagaland
369,29,December,NaN,kohima,nagaland
370,30,December,NaN,kohima,nagaland


In [16]:
df.rename(columns={"Date":"date","value":"aqi","variable":"month"},inplace=True)

In [17]:
df.to_csv("aqi_india_cities_2024")

# Deprecated, dictionary of cities by state.

In [18]:
cities_by_state = {}

In [19]:
# Use os to loop over every file in the directory.
for state in os.listdir("pollution_cities"):
    if state != ".DS_Store":
        
        # Go through every city in each state.
        name_list = []
        for city in os.listdir(f"pollution_cities/{state}"):
            if city != ".DS_Store" and city != ".ipynb_checkpoints":
                # For each city extract the name, using the function defined above.
                #print(city,state)
                name_list.append(city_name(city))
        # After completing the list of cities in state put into dictionary.
        cities_by_state[state] = name_list
                